In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Define paths
train_dir = 'Dataset_split/train'
val_dir = 'Dataset_split/val'
test_dir = 'Dataset_split/test'

# Model parameters
input_shape = (224, 224, 1)
batch_size = 32
epochs = 10
learning_rate = 1e-3

# Simple CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model with a specific learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Data generators
train_data_gen = ImageDataGenerator(
    rescale=1 / 255.0,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)

val_data_gen = ImageDataGenerator(
    rescale=1 / 255.0,
)

test_data_gen = ImageDataGenerator(
    rescale=1 / 255.0,
)

train_generator = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary'
)

val_generator = val_data_gen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary',
    shuffle = False
)

test_generator = test_data_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary',
    shuffle=False
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Found 14000 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.
Found 3000 images belonging to 1 classes.
Epoch 1/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 223s 502ms/step - accuracy: 0.7951 - loss: 0.3748 - val_accuracy: 0.9980 - val_loss: 0.0102
Epoch 2/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 356s 810ms/step - accuracy: 0.9896 - loss: 0.0318 - val_accuracy: 0.9970 - val_loss: 0.0148
Epoch 3/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 442s 1s/step - accuracy: 0.9946 - loss: 0.0215 - val_accuracy: 0.9967 - val_loss: 0.0106
Epoch 4/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 228s 518ms/step - accuracy: 0.9957 - loss: 0.0119 - val_accuracy: 0.9990 - val_loss: 0.0080
Epoch 5/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 223s 507ms/step - accuracy: 0.9969 - loss: 0.0127 - val_accuracy: 0.9990 - val_loss: 0.0053
Epoch 6/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 249s 566ms/step - accuracy: 0.9978 - loss: 0.0075 - val_accuracy: 0.9983 - val_loss: 0.0044
Epoch 7/10
438/438 ━━━━━━━━━━━━━━━━━━━━ 275s 627ms/step - accuracy: 0.9973 - loss: 0

In [16]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 22s 226ms/step - accuracy: 0.4988 - loss: 13.0506
Test Loss: 13.1124
Test Accuracy: 0.4997


In [15]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize the data generator for validation data
valid_datagen = ImageDataGenerator(rescale=1 / 255.0)

valid_generator = valid_datagen.flow_from_directory(
    directory="Dataset_split/val",
    target_size=(224, 224),
    color_mode="grayscale",
    class_mode="binary",
    batch_size=32,
    shuffle=False
)

steps_per_epoch = np.ceil(valid_generator.samples / valid_generator.batch_size).astype(int)

# Get predictions from the model
predictions = model.predict(valid_generator, steps=steps_per_epoch)

# Since this is a binary classification, we threshold predictions at 0.5
predicted_classes = (predictions > 0.5).astype(int).reshape(-1)

# Get the true classes
true_classes = valid_generator.classes

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))

# Print the classification report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=list(valid_generator.class_indices.keys())))

Found 3000 images belonging to 2 classes.


c:\Users\JJ\anaconda3\envs\JJ\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


94/94 ━━━━━━━━━━━━━━━━━━━━ 15s 160ms/step
Confusion Matrix:
[[1497    3]
 [   1 1499]]

Classification Report:
              precision    recall  f1-score   support

      Drowsy       1.00      1.00      1.00      1500
  Non Drowsy       1.00      1.00      1.00      1500

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

